In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time as time_lib

import numba as nb
from numbalsoda import lsoda_sig, lsoda, dop853
from numba import njit, cfunc

from scripts.dynamics import vecField
from scripts.utils import getCoarse,getNextCoarse
from scripts.plotting import plot_results

In [2]:
system = "Burger"

n_x = 10
n_t = 100

LB = -1.
UB = 1.
L = 50

weight = np.random.uniform(low=LB,high=UB,size=(L))
bias = np.random.uniform(low=LB,high=UB,size=(L))

vecRef = vecField(system=system)
if system=="Rober":
        k1,k2,k3 = vecRef.k1,vecRef.k2,vecRef.k3
        t_max = 100.
elif system=="SIR":
        beta,gamma,N = vecRef.beta,vecRef.gamma,vecRef.N
        t_max = 100.
elif system=="Brusselator":
        A,B = vecRef.A,vecRef.B 
        t_max = 12.
elif system=="Arenstorf":
        a,b = vecRef.a,vecRef.b 
        t_max = 17.0652165601579625588917206249 #One period
elif system=="Lorenz":
        sigma,r,b = vecRef.sigma,vecRef.r,vecRef.b
        t_max = 10.
elif system=="Burger":
        nu = vecRef.nu
        dx = vecRef.dx
        x = vecRef.x
        t_max = 1.
else:
        print("Dynamics not implemented")
        
time = np.linspace(0,t_max,int(t_max)+1)
dts = np.diff(time)

#### Setting the initial conditions

In [3]:
if system=="Rober":
        y0 = np.array([1.,0.,0])
elif system=="SIR":
        y0 = np.array([0.3,0.5,0.2])
elif system=="Brusselator":
        y0 = np.array([0.,1.])
elif system=="Arenstorf":
        y0 = np.array([0.994,0,0.,-2.00158510637908252240537862224])
elif system=="Lorenz":
        y0 = np.array([20.,5,-5])
elif system=="Burger":
        y0 = np.sin(2*np.pi*vecRef.x)
else:
        print("Dynamics not implemented")

In [4]:
data = {"LB" : LB,
        "UB" : UB,
        "L" : L,
        "y0" : y0,
        "weight" : weight,
        "bias" : bias,
        "n_x" : n_x,
        "n_t" : n_t,
        "system" : system}

In [5]:
@cfunc(lsoda_sig)
def f(t,y,dy,p):
    if system=="Rober":
        dy[0] =  -k1*y[0] + k3 * y[1] * y[2]
        dy[1] = k1*y[0] - k3 * y[1] * y[2] - k2*y[1]**2
        dy[2] = k2*y[1]**2
    elif system=="SIR":
        dy[0] = -beta*y[1]*y[0]/N
        dy[1] = beta*y[0]*y[1]/N - gamma * y[1]
        dy[2]= gamma * y[1]
    elif system=="Brusselator":
        dy[0] = A+y[0]**2*y[1]-(B+1)*y[0]
        dy[1] = B*y[0]-y[0]**2*y[1]
    elif system=="Arenstorf":
        mu = a
        x, y, vx, vy = y[0],y[2],y[1],y[3]
        D1 = ((x + a)**2 + y**2)**1.5
        D2 = ((x - b)**2 + y**2)**1.5
        dy[0] = vx
        dy[1] = x + 2*vy - b*(x + a)/D1 - a*(x - b)/D2
        dy[2] = vy
        dy[3] = y - 2*vx - b*y/D1 - a*y/D2
    elif system=="Lorenz":
        dy[0] = -sigma * y[0] + sigma * y[1]
        dy[1] = -y[0]*y[2]+r*y[0]-y[1]
        dy[2] = y[0]*y[1]-b*y[2]
    elif system=="Burger":
        y_array = nb.carray(y, (50,))
        #dy = nb.carray(dy, (50,))

        # We can now use y_array and dy_array as normal Numpy arrays
        # Compute the derivatives, omitting boundaries since they are homogeneous
        for i in range(1, 50-1):
            y_x = (y_array[i+1] - y_array[i-1]) / (2 * dx)
            y_xx = (y_array[i+1] - 2 * y_array[i] + y_array[i-1]) / dx**2
            dy[i] = -y_array[i] * y_x + nu * y_xx

        # Boundary conditions
        dy[0] = 0.
        dy[-1] = 0.0
    else:
        print("Dynamics not implemented")

In [6]:
funcptr = f.address # address to ODE function

In [7]:
@njit(parallel=True)
def fine_integrator(ics,dts):
    usol = np.zeros_like(ics)
    for i in nb.prange(len(ics)):
        u0 = ics[i]
        uu,_ = lsoda(funcptr, u0, t_eval=np.array([0,dts[i]]), rtol = 1e-6, atol = 1e-6)
        usol[i] = uu[-1]
    return usol

In [8]:
from scipy.integrate import solve_ivp

max_it = 10 #maximum number of parareal iterates
tol = 1e-5
computational_times_per_iterate = []
it = 0
is_converged = False

networks = []

initial_full = time_lib.time()

while it<max_it and is_converged==False:

    norm_difference = []

    if it==0:
        initial_time = time_lib.time()
        coarse_approx = getCoarse(previous=[],time=time,data=data,networks=networks)
        coarse_values_parareal = coarse_approx.copy()
        computational_times_per_iterate.append(time_lib.time()-initial_time)
            
    else:
        initial_time = time_lib.time()
        coarse_approx = getCoarse(previous=coarse_values_parareal,data=data,time=time,networks=networks)
        
        start_fine = time_lib.time()
        
        fine_int = fine_integrator(coarse_values_parareal,dts)
        print("Time required for the fine solver : ",time_lib.time()-start_fine)
        for i in range(len(time)-1):     
            previous = coarse_values_parareal[i+1].copy()
            coarse_values_parareal[i+1] = fine_int[i] + getNextCoarse(y=coarse_values_parareal[i],i=i,time=time,data=data,networks=networks) - coarse_approx[i+1]
            norm_difference = np.linalg.norm(coarse_values_parareal[i+1]-previous,2)
            
        computational_times_per_iterate.append(time_lib.time()-initial_time)
        print("Maximum norm of difference :",np.round(np.max(norm_difference),10))
        is_converged = np.max(norm_difference)<tol
        
    it+=1
    print(f"Iterate {it} completed")
    print(f"Time for iterate {it} is {computational_times_per_iterate[-1]}")  

total_time = time_lib.time()-initial_full

KeyboardInterrupt: 

In [ ]:
def get_detailed_solution():
    time_plot = np.linspace(0,networks[0].dt,101)
    sol = networks[0].plotOverTimeRange(time_plot)
    total_time = time_plot
    for i in np.arange(1,len(networks)):
        time_plot = np.linspace(0,networks[i].dt,101)[1:]
        sol = np.concatenate((sol,networks[i].plotOverTimeRange(time_plot)),axis=1)
        total_time = np.concatenate((total_time,time_plot+total_time[-1]),axis=0)
    return sol,total_time

In [ ]:
network_sol, time_plot = get_detailed_solution()

In [ ]:
initial = time_lib.time()
output, _ = dop853(funcptr, y0, t_eval=time_plot, rtol=1e-11, atol=1e-10)
final = time_lib.time()
print(f"Computational time : {final-initial}")

In [ ]:
if len(y0)==2:
    list_of_labels = [r"$x$",r"$y$"]
elif len(y0)==3:
    list_of_labels = [r"$x$",r"$y$",r"$z$"]
elif len(y0)==4:
    list_of_labels = [r"$x$",r"$\dot{x}$",r"$y$",r"$\dot{y}$"]

In [ ]:
output, _ = lsoda(funcptr, y0, t_eval=time_plot, rtol=1e-8, atol=1e-8)
plot_results(y0,system,time_plot,output,network_sol,list_of_labels,total_time,time,n_x,n_t,L)